In [4]:
import github_command as gt

In [52]:
gt.push(file_to_transfer='TD7_Image_Captioning_CNN_RNN.ipynb',
       message="initial commit",
       repos="TDs_ESILV.git")

# Image captionning

### CNN Network part
#### Get the InceptionV3 model trained on imagenet data

In [54]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
model = InceptionV3(weights='imagenet')
# Remove the last layer (output softmax layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [56]:
model_new.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, None, None, 3 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, None, None, 3 0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

### Data part

#### Image descriptions


In [38]:
folder_proj_path="/Users/lucbertin/Downloads/demos/flickr30k_images/"
captions_file = "results.csv"

In [39]:
import subprocess
subprocess.check_output(["head", "-n", "2", folder_proj_path+captions_file])

b'image_name| comment_number| comment\n1000092795.jpg| 0| Two young guys with shaggy hair look at their hands while hanging out in the yard .\n'

In [41]:
import pandas as pd
## Open descriptions dataset and append corresponding images
df = pd.read_csv(folder_proj_path+captions_file, sep='|')
df.head()

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [43]:
print(df.shape)
df.columns = df.columns.str.replace(' ', '')
print(df.columns)

(158915, 3)
Index(['image_name', 'comment_number', 'comment'], dtype='object')


In [44]:
## Appending startseq and endseq to each comment
df['comment2'] = ("startseq "  +  df.comment
                                        .str.lower()
                                        .str.split().str.join(" ")
                                        .str.replace(r"[^a-z0-9 ]", "") + " endseq")
df[df["comment2"].apply(lambda x: type(x)==float)]

,image_name,comment_number,comment,comment2
19999,2199200615.jpg,4 A dog runs across the grass .,NaN,NaN


In [45]:
#Lol
df.drop(19999, axis=0, inplace=True)

In [46]:
## Just take words occuring at least 10 times
#len(df.comment2.str.split(' ').tolist())
from collections import Counter
all_words = [item for sublist in df.comment2.str.split(' ').tolist() for item in sublist]
more_than_10_occurences = {k:val for k,val in Counter(all_words).items() if val>=10}
#more_than_10_occurences
df['comment2'] = df['comment2'].str.split(" ").apply(lambda val: [x for x in val if more_than_10_occurences.get(x) is not None])
len(more_than_10_occurences)

5464

In [50]:
TARGET_SIZE=(299,299)

In [53]:
def encode(image, model_new):
    """ Function to encode a given image into a vector of size (2048, ) using inceptionV3 """
    from keras.applications.inception_v3 import preprocess_input
    import numpy as np
    image = np.array(image) # transform img to array
    image = np.expand_dims(np.array(x), axis=0) # add one dimension for batch (keras needs it)
    image = preprocess_input(image) # preprocess the image for inceptionV3
    fea_vec = model_new.predict(image) # The model beign trained already, get the encoding vector for the image after a forward pass
    fea_vec = np.reshape(fea_vec, -1) # reshape from (1, 2048) to (2048, )
    return fea_vec

def load_batch_of_images(df, batch_size, model_for_encoding, folder_imgs_path):
    """ This function will only load batch_size pictures at a time (for computations)"""
    from PIL import Image as Im
    import pandas as pd
    # take a sample from the main dataset
    df_sub = df.sample(n=batch_size).reset_index(drop=True)
    # open corresponding images in new column
    df_sub['image'] = df_sub.image_name.apply( lambda x: Im.open(folder_imgs_path+x).resize(TARGET_SIZE))
    # transform to array, preprocess and encode images
    df_sub['image'] = df_sub.image.apply(lambda x: encode(x, model_for_encoding))
    return df_sub

def create_dictionnaries_for_string_convertion(vocab):
    """ Create an index to word dictionnary and a word to index one """
    ixtoword, wordtoix = {}, {}
    ix = 1
    for w in vocab:
        wordtoix[w] = ix
        ixtoword[ix] = w
        ix += 1
    return ixtoword, wordtoix

ixtoword, wordtoix = create_dictionnaries_for_string_convertion(vocab=more_than_10_occurences)
maximum_length_caption_on_all_dataset = max(df.comment2.apply(len)) # max caption length ( for homogeneity of input vectors )
maximum_length_caption_on_all_dataset

%time load_batch_of_images(df, 32, model_new, folder_imgs_path=folder_proj_path)

ValueError: Error when checking input: expected input_2 to have 4 dimensions, but got array with shape (1, 1)